# Low-Level API

## Prerequisites

If you've already completed the instructions on the **Installation** page, then let's get started.

In [13]:
import aiqc
from aiqc import datum

## 1. Ingest a `Dataset`

### Object Relational Model (ORM)

At the moment, AIQC supports the following types of Datasets:

* Single-file tabular/ flat/ delimited datasets.
* Multi-file image datasets

End users only need to worry about passing the right inputs to the Dataset class, but there are a few objects doing the legwork beneath the hood:

* `Dataset` ORM class with subclasses of either `Tabular` or `Image`.
  * `File` ORM class one or more files with subclasses of either `Tabular` or `Image`.
    * Dedicated `Tabular` and `Image` ORM classes for attributes specific to those data types (e.g. dtype mappings for flat files and colorscale for images).

> *Considering these types in the future:*
> * *Sequence/ time series: multi-file tabular (e.g. 3D numpy, HDF5)*
> * *Graph: multi-file nodes and multi-file edges (e.g. DGL)*

### Persisting and Compressing Structured Data

By default the actual bytes of the file are persisted to the SQLite `BlobField`. It gets gzip compressed, reducing the size by up to 90%. Maximum BlobField size is 2.147 GB, but once you factor in compression, your bottleneck is more likely to be memory beyond that size. The bytes themselves are Parquet (single-partitioned) because, using the PyArrow engine, it preserves every dtype except certain datetimes (which are honestly better off parsed into floats/ ordinal ints). Parquet is also integrated nicely into both Spark and Dask; frameworks for distributed, in-memory computation.

Persisting the file ensures reproducibility by: (a) keeping the data packaged alongside the experiment, and (b) helping entry-level users move away from relying on mutable dataframes they have had in-memory for extended periods of time or floating around on shared file systems.

> *However, we realize that a different approach will be required at scale, so the `source_path` of the file is recorded whenever possible. In the future we could just read the data from that path (e.g. NFS, RDBMS, HDFS, S3) if the BlobField is none. Or just switch our data fetching/ filtering to Dask because it uses the Pandas API and Parquet.*

### Data Sources

You can make a dataset from either:
* `Dataset.Tabular`
  * In-memory data structures (pandas dataframe, numpy array).
  * Flat files (csv, tsv, parquet).
    * Accepts urls.


* `Dataset.Image`
  * Any image file format that can be read by the Pillow library.
    * Accepts urls.

#### `Dataset.Tabular.from_pandas()`

In [3]:
df = datum.to_pandas('iris.tsv')

dataset = aiqc.Dataset.Tabular.from_pandas(
	dataframe = df
    , name = 'tab separated plants'
    , dtype = None #passed to pd.Dataframe(dtype)/ inferred
    , column_names = None #passed to pd.Dataframe(columns)
)

> Optionally, `dtype`, as seen in [`pandas.DataFrame.astype(dtype)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html), can be specified as either a single type for all columns, or as a dictionary that maps a specific type to each column name. This encodes features for analysis. We read NumPy into Pandas before persisting it, so `columns` and `dtype` are read directly by `pd.DataFrame()`.

#### `Dataset.Tabular.from_numpy()`

Must be a 2D NumPy N-Dimensional Array.

> *In the future, we may add support for ingesting 3D arrays as multi-file sequences.*

Regular *ndarrays* don't have column names, and I didn't like the API for *structured arrays* so you have to pass in columns names as a list. If you don't then column names will be numerically assigned in ascending order (zero-based index), but I didn't like the range object, so I stringified numerically assigned columns to string-based numbers.

In [4]:
arr =  df.to_numpy()
cols = list(df.columns)

dataset = aiqc.Dataset.from_numpy(
	ndarray = arr
    , name = None
    , dtype = None #passed to pd.Dataframe(dtype)/ inferred 
    , column_names = cols #passed to pd.Dataframe(columns)
)

#### `Dataset.Tabular.from_path`

Intended for flat files, delimited text, and structured tabular data. It's read in via Pandas, so it supports URLs to raw data and bytes as well.

In [ ]:
file_path = datum.get_path('iris_10x.tsv')

# We'll keep this larger dataset handy for `Foldset` creation later.
big_dataset = aiqc.Dataset.Tabular.from_path(
    file_path = file_path
    , source_file_format = 'tsv'
    , name = None
    , dtype = None
    , column_names = None
    , skip_header_rows = 'infer' #passed to `pd.read_csv(header)`. Incompatible w Parquet.
)

> If you leave `name` blank, it will default to a human-readble timestamp with the appropriate file extension (e.g. '2020_10_13-01_28_13_PM.tsv').

#### `Dataset.Image.from_urls()`

During ingestion, all image files must have the same `Image.mode` and `Image.size` according to the Pillow library.

> https://pillow.readthedocs.io/en/stable/handbook/concepts.html

Image datasets are somewhat multi-modal in that, in order to perform supervised learning, they require a loosely coupled `Dataset.Tabular` that contains their labels.

In [ ]:
df = datum.to_pandas(name='brain_tumor.csv')
df.head()

The `['status']` column of this dataframe serves as the Label of that sample. We'll construct a `Dataset.Tabular` from this.

In [ ]:
tabular_dataset = aiqc.Dataset.Tabular.from_pandas(dataframe=df)

In [ ]:
tabular_label = tabular_dataset.make_label(columns=['status'])

Meanwhile, the `['url']` column acts as a manifest in that it contains the URL of the image file for that sample, solely for the purposes of initial ingestion. We'll construct a `Dataset.Image` from this.

In [ ]:
image_urls = datum.get_remote_urls(manifest_name='brain_tumor.csv')

In [ ]:
image_dataset = aiqc.Dataset.Image.from_urls(urls=image_urls)

In [ ]:
image_featureset = image_dataset.make_featureset()

Skipping forward a bit, we bring the heterogenous `Featureset` and `Label` together in the `Splitset`, and they can be used as normal. You can even construct a `Foldset` from this splitset.

In [ ]:
image_splitset = image_featureset.make_splitset(
    label_id = tabular_label.id
    , size_test = 0.24
    , size_validation = 0.12
)

#### `Dataset.Image.from_folder()`

This method works in the same was as `from_urls` except that it accepts the path of a single, local directory rather than a list of urls to individual files.

When reading in the image files in that folder, the fantastic `natsort.natsorted` library is used as the source of truth for the ordering of the files. When Python reads a list of files from a directory, it does so by insertion order rather than alpha-numerically. So make sure your tabular manifest has the same order.

> https://natsort.readthedocs.io/en/master/api.html#natsort.natsorted

### Reading Datasets

All of the sample-related objects in the API have `to_numpy()` and `to_pandas()` methods that accept the following arguments:

* `samples=[]` list of indeces to fetch.
* `columns=[]` list of columns to fetch.
* In some cases you can specify a `split`/ `fold` name.

For structured data, since the `Dataset` itself is fairly removed from the `File.Tabular` it creates, you can get that tabular file with `Dataset.Tabular.get_main_tabular(dataset_id)` to inspect attributes like `dtypes` and `columns`.

Later, we'll see how these arguments allow downstream objects like `Splitset` and `Foldset` to slice up the data.

### `Dataset.Tabular.to_pandas()`

In [6]:
df = dataset.to_pandas()
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [100]:
df = aiqc.Dataset.to_pandas(
    id = dataset.id 
    , samples = [0,13,29,79]
    , columns = ['sepal_length', 'sepal_width']
)
df.tail()

,sepal_length,sepal_width
0,5.1,3.5
13,4.3,3.0
29,4.7,3.2
79,5.7,2.6


### `Dataset.Tabular.to_numpy()`

In [99]:
arr = dataset.to_numpy(
    samples = [0,13,29,79] 
    , columns = ['petal_length', 'petal_width']
)
arr[:4]

array([[1.4, 0.2],
       [1.1, 0.1],
       [1.6, 0.2],
       [3.5, 1. ]])

In [9]:
arr = aiqc.Dataset.to_numpy(id=dataset.id)
arr[:4]

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ]])

### `Dataset.Image.to_pillow()`

Returns a list of `Pillow.Image`'s. You can actually see the image when you call them. 

images_pillow = image_dataset.to_pillow(samples=[1,2,3])
images_pillow[0]

### `Dataset.Image.to_numpy()`

This simply performs `np.array(Pillow.Image)`. Returns an N-dimensional array where the dimensions vary based on the `mode` aka colorscale of the image. For example, it returns '3D of 2Ds for black and white' or '4D of 3Ds for colored' - which would change the class of convultional layer you would use (`Conv1D`:`Conv3D`).



images_arr = image_dataset.to_numpy(samples=[1,2,3])
images_arr.shape

> At the moment, we haven't found it necessary to provide a `to_pandas` method for images as they have no need for column names, the dtypes are homogenous, images are used as a whole so there is no filtering, Pandas isn't great with 3D data, and Pillow is integrated with NumPy.

## 2. Select the `Label` column(s).

### ORM

From a Dataset, pick the column(s) that you want to predict/ train against. Creating a `Label` won't duplicate your data! It simply marks the Dataset `columns` to be used for supervised learning. 

Later, we'll see that a `Label` triggers:
* The `supervision` attribute of a `Splitset` to be either `'unsupervised'`/`'supervised'`.
* Approval/ rejection of the `Algorithm.analysis_type`. For example, you wouldn't perform regression on a string label.

Part of the magic of this library is that it prevents you from making silly mistakes like these so that you aren't faced with some obscure NumPy/ Tensor, dtype/ dimensionality error on the nth layer of your neural network.

For categorical labels, but not for continuous/float labels, the `Label.unique_classes` are recorded.

### Deriving Labels

Keep the name of the label column handy as you may want to re-use it later when excluding features.

In [10]:
label_column = 'species'

Implicit IDs

In [11]:
label = dataset.make_label(columns=[label_column])

> `columns=[label_column]` is a list in case users have already OneHotEncoded (OHEd) their label. If multiple columns are provided, then they must already be in OHE format. I'm not keen on supporting multi-label/ simultaneous analysis, but that could changed based on feasibility and user demand.

Explicit IDs

In [12]:
label = aiqc.Label.from_dataset(
	dataset_id=1
	, columns=[label_column]
)

### Reading Labels

The `Label` comes in handy when we need to fetch what is traditionally referred to as '*Y*' in tutorials. It also accepts a `samples` argument, so that `Splitset` can subset it.

In [13]:
label.to_pandas().tail()

,species
145,2
146,2
147,2
148,2
149,2


In [14]:
label.to_numpy(samples=[0,33,66,99,132])[:5]

array([[0],
       [0],
       [1],
       [1],
       [2]])

## 3. Select the `Featureset` column(s).

### ORM

Creating a Featureset won't duplicate your data! It simply records the Dataset `columns` to be used as features during training. 

There are three ways to define which columns you want to use as features:

- `exclude_columns=[]` e.g. use all columns except the label column.
- `include_columns=[]` e.g. only use these columns that I think are informative.
- Leave both of the above blank and all columns will be used (e.g. images or unsupervised leanring).

For structured data, since the Featureset is far removed from the `File.Tabular` that it is derived from, there is a `Featureset.get_dtypes()` method. This will come in handy when we are selecting dtypes/columns to include/ exclude in our `Featurecoder`(s).

### Deriving Labels

Via `include_columns=[]`

In [16]:
include_columns = [
    'sepal_length',
    'petal_length',
    'petal_width'
]

In [17]:
featureset = dataset.make_featureset(include_columns=include_columns)

Via `exclude_columns=[]`

In [15]:
featureset = dataset.make_featureset(exclude_columns=[label_column])

In [18]:
featureset.columns

['sepal_length', 'petal_length', 'petal_width']

Either way, any excluded columns will be recorded since they are used for dropping.

In [19]:
featureset.columns_excluded

['sepal_width', 'species']

Again, for images, just perform `Dataset.Image.make_featureset()` since you'll likely want to include all pixels and your label column is in a separate, coupled Dataset.

### Reading Featuresets

In [20]:
featureset.to_numpy()[:4]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2],
       [4.6, 1.5, 0.2]])

In [21]:
featureset.to_pandas(samples=[0,16,32,64]).tail()

,sepal_length,petal_length,petal_width
0,5.1,1.4,0.2
16,5.4,1.3,0.4
32,5.2,1.5,0.1
64,5.6,3.6,1.3


## 4. Slice samples with a `Splitset`.

A `Splitset` divides a the samples of the Dataset into the following *splits* in the table below. It is the central object of the data preparation side of the ORM in that it touches `Label`, `Featureset`, `Foldset`, and `Encoderset`. It is the only mandatory data preparation object required by the training `Batch`.

Both contiuous and categorical `Labels` are automatically stratified.

| Split                 | Description                                                                                                                                                                                             |
|-----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| train                 | The samples that the model will be trained upon. <br/>Later, we’ll see how we can make *cross-folds from our training split*. <br/>Unsupervised learning will only have a training split.                 |
| validation (optional) | The samples used for training evaluation. <br/>Ensures that the test set is not revealed to the model during training.                                                                                  |
| test (optional)       | The samples the model has never seen during training. <br/>Used to assess how well the model will perform on unobserved, natural data when it is applied in the real world aka how generalizable it is. |

Again, creating a Splitset won't duplicate your data. It simply denotes the sample indeces (aka rows) to be used in the splits that you specify!

### Split Strategies

#### a) Default supervised 70-30 split.

If you only provide a Label, then 70:30 train:test splits will be generated.

In [22]:
splitset = featureset.make_splitset(label_id=label.id)

#### b) Specifying test size.

In [23]:
splitset = featureset.make_splitset(
	label_id = label.id
	, size_test = 0.30
)

#### c) Specifying validation size.

In [24]:
splitset = featureset.make_splitset(
	label_id = label.id
	, size_test = 0.20
	, size_validation = 0.12
)

#### d) Taking the whole dataset as a training split.

In [25]:
splitset_unsupervised = featureset.make_splitset()

> Label-based stratification is used to ensure equally distributed label classes for both categorical and continuous data.
>
> If you want more control over stratification of continuous splits, specify the number of `continuous_bin_count` for grouping.

#### e) Stratification of continuous labels.

All splits are stratified by default in that they contain similar distributions of unique label classes so that each split is a statistically accurate representation of the population as a whole.

In order to support this process for continuous labels, binning/ discretization is utilized. For example, if 4 bins are used, values from *0.0 to 1.0* would be binned as *[0.0-0.25, 0.25-0.50, 0.50-0.75, 0.75-1.0]*. This is controlled by the `make_splitset(bin_count:int)` argument. 

> Reference the handy `Pandas.qcut()`  and the source code `pd.qcut(x=array_to_bin, q=bin_count, labels=False, duplicates='drop')` for more detail.

### Reading Splitsets

In [ ]:
splitset.samples.keys()

`.keys()` of 1st layer are referred to as "split_name" in the source code: e.g. 'train' as well as, optionally, 'validation' and 'test'.
  
`Splitset.samples` on disk:
```
 {
     'train': [<sample_indeces>],
     'validation': [<sample_indeces>],
     'test': [<sample_indeces>]
 }
```

You can also verify the actual size of your splits.

In [26]:
splitset.sizes

{'validation': {'percent': 0.12, 'count': 18},
 'test': {'percent': 0.2, 'count': 30},
 'train': {'percent': 0.68, 'count': 102}}

The main attribute of the splitset is the `samples` dictionary. Again, on-disk this only contains sample indeces. The dictionary is structured like so:

### `Splitset.to_numpy()`

When fetched to memory, the `.keys()` of the 2nd layer are: 'features' and, optionally, 'labels'.

Note that if you do not specified neither a `size_validation` nor `size_test`, then your dictionary will contain neither a `['validation']` nor `['test']` split.

In [27]:
splitset.to_numpy()['train']['features'][:4]

array([[4.9, 4.5, 1.7],
       [6.4, 5.3, 1.9],
       [5.8, 5.1, 1.9],
       [4.8, 1.6, 0.2]])

### `Splitset.to_pandas()`

Getting more fine-tuned, both the numpy and pandas methods support a few optional filters for the sake of memory-efficiency when fetching larger splits.

For example, imagine you are fetching data to specifically encode the only float column in the featureset of the test split. You don't need the labels and you don't need the other columns.

In [ ]:
splitset.to_pandas(
	splits = ['test']
	, include_label = False
	, include_featureset = True
	, feature_columns = ['sepal_width']
)['test']['features'].head()

## 5. Optionally, create a `Foldset` for cross-validation.

### ORM

*Reference the [scikit-learn documentation](https://scikit-learn.org/stable/modules/cross_validation.html) to learn more about folding.*

![Cross Folds](../images/cross_fold.png)

We refer to the left out fold (blue) as the `fold_validation` and the remaining training data as the `folds_train_combined` (green).

> *In the future, we may introduce more folding `strategies` aside from leave-one-out.*

#### `Fold` objects

For the sake of determining which samples get trained upon, the only thing that matters is the slice of data that gets left out.

> Tip - DO NOT use a `Foldset` unless your *(total sample count / fold_count)* still gives you an accurate representation of your sample population. If you are ignoring that advice and stretching to perform cross-validation, then at least ensure that *(total sample count / fold_count)* is evenly divisible. Both of these tips help avoid poorly stratified/ undersized folds that perform either too well (only most common label class present) or poorly (handful of samples and a few inaccurate prediction on a normally good model).
>
> Tip - The sample indeces of the validation fold are not discarded. In fact, `fold_validation` can actually be used alongside a split `validation` for double validation 🤘. However, it's more sensible to skip the validation split when cross-validating because you'll want each `fold_validation` to be as large (representative of the population) as possible. Folds naturally have fewer samples, so a handful of incorrect predictions have the potential to offset your aggregate metrics.
> 
> Candidly, if you've ever performed cross-validation manually, let alone systematically, you'll know that, barring stratification of continuous labels, it's easy enough to construct the folds, but then it's a pain to generate performance metrics (e.g. `zero_division`, absent OHE classes) due to the absence of outlying classes and bins. Time has been invested to handle these scenarios elegantly so that folds can be treated as first-class-citizens alongside splits. That being said, if you try to do something undersized like "150 samples in their dataset and a `fold_count` > 3 with `unique_classes` > 4," then you may run into edge cases.

Similar to `Splitset.samples`, there is a `Fold.samples` dictionary of sample indeces with the following `.keys()`:
* `samples['folds_train_combined']` - all the included folds.
* `samples['fold_validation']` - the fold that got left out.

![cross fold objects](../images/cross_fold_objects.png)

### Deriving Foldsets

In [ ]:
big_label = big_dataset.make_label(columns=[label_column])
big_fset = big_dataset.make_featureset(exclude_columns=[label_column])
big_splits = big_fset.make_splitset(
	label_id = big_label.id
	, size_test = 0.30
)

Now we are ready to generate 5 `Fold` objects that belong to the `Foldset`.

In [30]:
foldset = big_splits.make_foldset(fold_count=5, bin_count=3)

In [31]:
list(foldset.folds)

[<Fold: 1>, <Fold: 2>, <Fold: 3>, <Fold: 4>, <Fold: 5>]

### Reading Foldsets

##### Sample indeces of each Fold:

In [32]:
foldset.folds[0].samples['folds_train_combined'][:10]

[0, 1, 2, 4, 5, 7, 9, 11, 12, 13]

In [33]:
foldset.folds[0].samples['fold_validation'][:10]

[3, 6, 8, 10, 21, 29, 32, 33, 38, 46]

### `Foldset.to_numpy()`

In order to reduce memory footprint the `to_numpy()` and `to_pandas()` methods introduce the `fold_index` argument.

If no fold_index is specified, then it will fetch all folds and give each fold a numeric key according to its index.

So you need to specify the `fold_index` as the first key when accessing the dictionary.

In [35]:
foldset.to_numpy(fold_index=0)[0]['fold_validation']['features'][:4]

array([[4.6, 3.1, 1.5, 0.2],
       [4.6, 3.4, 1.4, 0.3],
       [4.4, 2.9, 1.4, 0.2],
       [5.4, 3.7, 1.5, 0.2]])

### `Foldset.to_pandas()`

Similar to `splitset.to_numpy(splits:list)`, the `foldset.to_numpy(fold_names:list)` argument allows you to pluck the `['folds_train_combined]` and `['fold_validation]` slices. Just make sure you remember to specific all 3 levels of keys when accessing the result.

In [36]:
foldset.to_pandas(
    fold_index = 0
	, fold_names = ['folds_train_combined']
	, include_label = True
	, include_featureset = False
	, feature_columns = [label_column]
)[0]['folds_train_combined']['labels'].tail()

,species
1044,2
1046,2
1047,2
1048,2
1049,2


## 6. Optionally, stage an `Encoderset` for encoding.

### Background

Certain algorithms either (a) require features and/ or labels formatted a certain way, or (b) perform MUCH better when their values are normalized. For example:

* Converting ordinal or categorical string data `[dog, cat, fish]` into one-hot encoded format `[[1,0,0][0,1,0][0,0,1]]`.
* Scaling continuous features from (-1 to 1) or (0.0 to 1.0). Or transforming them to resemble a more Gaussian distribution.

There are two phases of encoding:
1. `fit` - where the encoder learns about the values of the samples made available to it. Ideally, you only want to `fit` aka learn from your training split so that you are not *"leaking"* information from your validation and test spits into your encoder!
2. `transform` - where the encoder transforms all of the samples in the population.

AIQC has solved the following challenges that encoding brings to light:
* How does one dynamically `fit` on only the training samples in advanced scenarios like cross-validation where a different fold is used for validation each time?
* For certain encoders, especially categorical ones, there is arguably no leakage. If an encoder is arbitrarilly assigning values/ tags to a sample through a process that is not aggregate-informed, then the information that is reveal to the `fit` is largely irrelevant. As an analogy, if we are examining swan color and all of a sudden there is a black swan... it's clearly not white, so slap a non-white label on it and move on. In fact, the prediction process and performance metric calucatlion may fail if it doesn't know how to handle the previously unseen category.
* Certain encoders only accept certain dtypes. Certain encoders only accept certain dimensionality (e.g. 1D, 2D, 3D) or shape patterns (odd-by-odd square). Unfortunately, there is not much uniformity here.
* Certain encoders output extraneous objects that don't work with deep learning libraries.

> *For now, only `sklearn.preprocessing` methods are supported. That may change as we add support for more low-level tensor-based frameworks like PyTorch.*

Keeping this in mind, we create an `Encoderset` for our `Splitset`. We can attach a `Labelcoder` and/ or `Featurecoder`(s).

In [ ]:
encoderset = splitset.make_encoderset()

And then import any scikit-learn encoders that you need. AIQC only supports the uppercase methods (e.g. `RobustScaler`, but not `robust_scale`) because the lowercase methods do not separate the `fit` and `transform` steps. FYI, most of the uppercase methods have a combined `fit_transform` method if you need them. 

> https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [37]:
from sklearn.preprocessing import *

## 7. Optionally, set a single `Labelcoder`.

### Background

The simplistic `Labelcoder` is a good warmup for the moe advanced `Featurecoder`.

Of course, you cannot encode Labels if your `Splitset` does not have labels in the first place.

The process is straightforward. You provide an instantiated encoder [e.g. `StandardScaler()` not `StandardScaler`], and then AIQC will:
* Verify that the encoder works with your `Label`'s dtype, sample values, and figure out what dimensionality it needs in order to succeed.
* Validate the attributes of your encoder to smooth out any common errors they would cause.
* Determine whether the encoder should be `fit` either (a) exclusively on the train split, or (b) if it is not prone to leakage, inclusively on the entire dataset thereby reducing the chance of errors arising.

In [ ]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = OneHotEncoder(sparse=False)
)

## 8. Optionally, determine a sequence of `Featurecoder`(s).

### Background

The `Featurecoder` has the same validation process as the `Labelcoder`. However, it is not without its own challenges:

* We want to be able to apply different encoders to features of different dtypes. So it's likely that the same encoder will neither be applied to all columns, nor will all encoders be applied at the same exact time.

* Additionally, even within the same dtype (e.g. float/ continuous), different distributions call for different encoders.

* Commonly used encoders such a `OneHotEncoder` can ouput multiple columns from a single column input. Therefore, the structure of the features columns is not fixed during encoding.

* And finally, throughout this entire process, we need to avoid data leakage.

For these reasons, `Featurecoder`'s are applied sequentially; in an ordered chain, one after the other. After an encoder is applied, its columns are removed from the raw featureset and placed into an intermediary cache specific to each split/ fold. 

> Right now, `Featurecoder` cannot be created for `Dataset.Image.Featureset`. I'm not opposed to changing this, but I would just have to account for 3D arrays.

### Filtering feature columns

The filtering mode is either:
* Inclusive (`include=True`) encode columns that match the filter.
* Exclusive (`include=False`) encode columns outside of the filter.

Then you can select:
1. An optional list of `dtypes`.
2. An optional list of `columns` name.
  * The column filter is applied after the dtype filter. 
  
> You can create a filter for all columns by setting `include=False` and then seting both `dtypes` and `columns` to `None`.

After submitting your encoder, if `verbose=True` is enabled:
* The validation rules help determine why it may have failed.
* The print statements help determine which columns your current filter matched, and which raw columns remain. 

In [ ]:
featurecoder = encoderset.make_featurecoder(
    sklearn_preprocess = PowerTransformer(method='yeo-johnson', copy=False)
    , include = True
    , dtypes = ['float64']
    , columns = None
    , verbose = True
)

You can also view this information via the following attributes: `matching_columns`, `leftover_dtypes`, and `leftover_columns`.

## 9. Create an `Algorithm` aka model.

### ORM

Now that our data has been prepared, we transition to the other half of the ORM where the focus is the logic that will be applied to that data.

> An `Algorithm` is the ORM's codename for a machine learning model since *Model* is the most important *reserved word* for ORMs.

The following attributes tell AIQC how to handle the Algorithm behind the scenes:
* `library` - right now, only 'keras' is supported.
  * Each library's model object and callbacks (history, early stopping) need to be handled differently.
  
  
* `analysis_type` - right now, these types are supported:
  * `'classification_multi'`, `'classification_binary'`, `'regression'`.
  * Used to determine which performance metrics to run.
  * Must be compatible with the type of label fed to it.

### Model Definition

The `Algorithm` is composed of the functions:
* `function_model_build`.
* `function_model_train`.
* `function_model_evaluate` (optional, inferred by `analyis_type`). 
* `function_model_predict` (optional, inferred by `analyis_type`).

> May provide overridable defaults for build and train in the future.

You can name the functions whatever you want, but do not change the predetermined arguments (e.g. `input_shape`,`**hyperparameters`, `model`, etc.).

As we define these functions, we'll see that we can pass *hyperparameters* into these function like so: `hyperparameters['<some_variable_name>']` using the `**hyperparameters` kwarg. Later, we'll provide a list of values for each entry in the hyperparameters dictionary.

Let's import the modules that we need.

In [40]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

> Later, when running your `Job`'s, if you receive a "module not found" error, then you can try troubleshooting by importing that module directly within the function where it is used.

#### Function to build model

If you normally don't wrap the building of your model in a function, don't be scared, all you have to do is add `return model` to the bottom of it, and AIQC will handle the rest. Also, if using `hyperparameters` feels intimidating, you can skip them.

The automatically provided `features_shape` and `label_shape` are handy because:
* The number of feature/ label columns is mutable due to encoders (e.g. OHE). 
* Shapes can be less obvious in multi-dimensional scenarios like colored images.

> You can customize the metrics if you so desire (e.g. change the loss or accuracy), but they will only be applied to the training process/ `History` callback. We'll see later that AIQC will calculate metrics for you automatically.

In [62]:
def function_model_build(features_shape, label_shape, **hyperparameters):
    
	model = Sequential()
	model.add(Dense(hyperparameters['neuron_count'], input_shape=features_shape, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.2))
	model.add(Dense(hyperparameters['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(input_shape=label_shape, activation='softmax'))

	opt = keras.optimizers.Adamax(hyperparameters['learning_rate'])
	model.compile(
		loss = 'categorical_crossentropy'
		, optimizer = opt
		, metrics = ['accuracy']
	)
	return model

#### Function to train model

* `samples_train` - the appropriate data will be fed into the training cycle. For example, `Foldset.samples[fold_index]['folds_train_combined']` or `Splitset.samples['train']`.

* `samples_evaluate` - the appropriate data is made available for evaluation. For example, `Foldset.samples[fold_index]['fold_validation']`, `Splitset.samples['validation']`, or `Splitset.samples['test']`.

In [63]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    
	model.fit(
		samples_train["features"]
		, samples_train["labels"]
		, validation_data = (
			samples_evaluate["features"]
			, samples_evaluate["labels"]
		)
		, verbose = 0
		, batch_size = 3
		, epochs = hyperparameters['epoch_count']
		, callbacks=[History()]
	)
    
	return model

##### Optional, callback to stop training early.

*Early stopping* isn't just about efficiency in reducing the number of `epochs`. If you've specified 300 epochs, there's a chance your model catches on to the underlying patterns early, say around 75-125 epochs. At this point, there's also good chance what it learns in the remaining epochs will cause it to overfit on patterns that are specific to the training data, and thereby and lose it's simplicity/ generalizability.

> The `val_` prefix refers to the evaluation samples.
>
> Remember, regression does not have accuracy metrics.
>
> `TrainingCallback.Keras.MetricCutoff` is a custom class we wrote to make multi-metric cutoffs easier, so you won't find information about it in the official Keras documentation.

In [ ]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    # Define one or more metrics to monitor.
    metrics_cuttoffs = [
        {"metric":"val_accuracy", "cutoff":0.9, "above_or_below":"above"},
        {"metric":"val_loss", "cutoff":0.2, "above_or_below":"below"}
    ]
    cutoffs = aiqc.TrainingCallback.Keras.MetricCutoff(metrics_cuttoffs)
    # Remember to append `cutoffs` to the list of callbacks.
    callbacks=[History(), cutoffs]
    
    # No changes here.
	model.fit(
		samples_train["features"]
		, samples_train["labels"]
		, validation_data = (
			samples_evaluate["features"]
			, samples_evaluate["labels"]
		)
		, verbose = 0
		, batch_size = 3
		, epochs = hyperparameters['epoch_count']
		, callbacks = callbacks
	)
    
	return model

#### Optional, function to predict samples

`function_model_predict` will be generated for you automatically if set to `None`. The `analysis_type` and `library` of the Algorithm help determine how to handle the predictions.

##### a) Regression default.

In [64]:
def function_model_predict(model, samples_predict):
    predictions = model.predict(samples_predict['features'])
    return predictions

##### b) Classification binary default.

All classification `predictions`, both mutliclass and binary, must be returned in ordinal format. 

> For most libraries, classification algorithms output *probabilities* as opposed to actual predictions when running `model.predict()`. We want to return both of these object `predictions, probabilities` (the order matters) to generate performance metrics behind the scenes.

In [65]:
def function_model_predict(model, samples_predict):
    probabilities = model.predict(samples_predict['features'])
    # This is the official keras replacement for binary classes `.predict_classes()`
    # It returns one array per sample: `[[0][1][0][1]]` 
    predictions = (probabilities > 0.5).astype("int32")
    
    return predictions, probabilities

##### c) Classification multiclass default.

In [66]:
def function_model_predict(model, samples_predict):
    import numpy as np
    probabilities = model.predict(samples_predict['features'])
    # This is the official keras replacement for multiclass `.predict_classes()`
    # It returns one ordinal array per sample: `[[0][2][1][2]]` 
    predictions = np.argmax(probabilities, axis=-1)
    
    return predictions, probabilities

#### Optional, function to calculate loss

When creating an `Algorithm`, the evaluate function will be generated for you automatically if set to `None`. The `analysis_type` and `library` of the Algorithm help determine how to handle the predictions.

The only trick thing here is when `keras.metrics` returns multiple metrics, like *accuracy* and/ or *R^2*. All we are after in this case is the loss for the split/ fold in question.

In [67]:
def function_model_loss(model, samples_evaluate):
    metrics = model.evaluate(samples_evaluate['features'], samples_evaluate['labels'], verbose=0)
    if (isinstance(metrics, list)):
        loss = metrics[0]
    elif (isinstance(metrics, float)):
        loss = metrics
    else:
        raise ValueError(f"\nYikes - The 'metrics' returned are neither a list nor a float:\n{metrics}\n")
    return loss

> In contrast to openly specifying a loss function, for example `keras.losses.<loss_fn>()`, the use of `.evaluate()` is consistent because it comes from the compiled model. Also, although `model.compiled_loss` would be more efficient, it requires making encoded `y_true` and `y_pred` available to the user, whereas `.evaluate()` can be called with the same arugments as the other `function_model_*` and many deep learning libraries support this approach. 

#### Group the functions together in an `Algorithm`!

In [ ]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
	, analysis_type = "classification_multi"
	, function_model_build = function_model_build
	, function_model_train = function_model_train
	, function_model_predict = function_model_predict
	, function_model_loss = function_model_loss
)

> <!> Remember to use `make` and not `create`. Deceptively,  `create` runs because it is a standard, built-in ORM method. However, it does so without any validation logic.

## 8. Optional, associate a `Hyperparamset` with your model.

The `hyperparameters` below will be automatically fed into the functions above as `**kwargs` via the `**hyperparameters` argument we saw earlier.

For example, wherever you see `hyperparameters['neuron_count']`, it will pull from the *key:value* pair `"neuron_count": [9, 12]` seen below. Where "model A" would have 9 neurons and "model B" would have 12 neurons.

In [69]:
hyperparameters = {
	"neuron_count": [9, 12]
	, "epoch_count": [30, 60]
    , "learning_rate": [0.01, 0.03]
}

hyperparamset = aiqc.Hyperparamset.from_algorithm(
	algorithm_id = algorithm.id
	, hyperparameters = hyperparameters
)

> The number of unique combinations escalates quickly, so in the future, we will provide different strategies for generating and selecting parameters to experiment with. 

#### `Hyperparamcombo` objects.

Each unique combination of hyperparameters is recorded as a `Hyperparamcombo`.

Ultimately, a training `Job` will be constructed for each unique combinanation of hyperparameters aka `Hyperparamcombo`.

In [70]:
hyperparamset.hyperparamcombo_count

8

In [71]:
hyperparamcombos = hyperparamset.hyperparamcombos

for h in hyperparamcombos:
    print(h.hyperparameters)

{'neuron_count': 9, 'epoch_count': 30, 'learning_rate': 0.03}
{'neuron_count': 9, 'epoch_count': 30, 'learning_rate': 0.05}
{'neuron_count': 9, 'epoch_count': 60, 'learning_rate': 0.03}
{'neuron_count': 9, 'epoch_count': 60, 'learning_rate': 0.05}
{'neuron_count': 12, 'epoch_count': 30, 'learning_rate': 0.03}
{'neuron_count': 12, 'epoch_count': 30, 'learning_rate': 0.05}
{'neuron_count': 12, 'epoch_count': 60, 'learning_rate': 0.03}
{'neuron_count': 12, 'epoch_count': 60, 'learning_rate': 0.05}


## 9. Create a `Batch` of training `Jobs`.

The `Batch` is the central object of the "logic side" of the ORM. It ties together everything we need for training and hyperparameter tuning.

In [79]:
batch = aiqc.Batch.from_algorithm(
	algorithm_id = algorithm.id
	, splitset_id = splitset.id
	, hyperparamset_id = hyperparamset.id # Optional.
	, foldset_id = None # Optional.
	, encoderset_id = encoderset.id # Optional.
    , repeat_count = 3
)

* `repeat_count:int` allows us to run the same `Job` multiple times. Normally, each `Job` has 1 `Result` object associated with it upon completion. However, when `repeat_count` is used, a single `Job` will have multiple `Results`.

> Due to the fact that training is a *nondeterministic* process, we are likely to get different results each time we train a model, even if we use the same set of parameters. Perhaps you've have the right topology and parameters, but, this time around, the model just didn't recgonize the patterns. Similar to flipping a coin, there is a degree of chance in it, but the real trend averages out upon repetition. 

* `hide_test:bool` excludes the test split from the performance metrics and visualizations. This avoids data leakage by forcing the user to make decisions based on the performance on their model on the training and evaluation samples.

### `Job` objects.

Each `Job` in the Batch represents a `Hyperparamcombo` that needs to be trained.

> If a `Foldset` is used during `Batch` creation, then (a) the number of jobs is multiplied by the `hyperparamcombo_count` and the `fold_count`, (b) each Job will have a `Fold`. Additionally, a superficial `Jobset` will be used to keep track of all Jobs related to that Foldset.

`Job.status` keeps track of the execution phase: "Not yet started", "Queued", or "Succeeded"

In [80]:
batch.jobs[0].status

'Not yet started'

In [81]:
batch.get_statuses()

{17: 'Not yet started',
 18: 'Not yet started',
 19: 'Not yet started',
 20: 'Not yet started',
 21: 'Not yet started',
 22: 'Not yet started',
 23: 'Not yet started',
 24: 'Not yet started'}

### Execute all `Jobs`.
The Job queue is asynchronously executed as a background multi-process, so that you can continue to write code on the main process. You can poll the Job `status` as seen above, but there will be a progress bar that updates after each model is done training.

In [82]:
batch.run_jobs()

🔮 Training Models 🔮:  25%|██████████▌                               | 2/8 [00:04<00:12,  2.14s/it]

The queue is interuptable. You can stop the execution of a batch and resume it later.

> This also comes in handy if either your machine or Python kernel either crash or are accidentally interupted. During stress tests, I observed that sometimes when running multiple batches at the same time, the SQLite database would lock against one process due to simultaneous writes. Whatever the reason, rest easy, just `run_jobs()` again to pick up where you left off!

In [86]:
# Just giving the batch some time to actually start in this notebook.
from time import sleep
sleep(5)

In [83]:
batch.stop_jobs()


Killed `multiprocessing.Process` 'aiqc_batch_3' spawned from Batch <id:3>



In [84]:
batch.get_statuses()

{17: 'Succeeded',
 18: 'Succeeded',
 19: 'Queued',
 20: 'Queued',
 21: 'Queued',
 22: 'Queued',
 23: 'Queued',
 24: 'Queued'}

In [85]:
batch.run_jobs()


Resuming jobs...



🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:16<00:00,  2.12s/it]


## 10. Assess the `Results`.

Each `Job` has a `Result`. The following attributes are automatically written to the `Result` after training.
    
* `model_file`: HDF5 bytes of the model.
* `history`: per epoch metrics recorded during training.
* `predictions`: dictionary of predictions per split/ fold.
* `probabilities`: dictionary of prediction probabilities per split/ fold.
* `metrics`: dictionary of single-value metrics depending on the analysis_type.
* `plot_data`: metrics readily formatted for plotting.

> The dictionary attributes use split/ fold-based keys.

> For more information of visualization of performance metrics, reference the **Visualization & Metrics** documentation.

### Fetching the trained model.

In [90]:
compiled_model = batch.jobs[0].results[0].get_model()
compiled_model

### Fetching metrics.

In [92]:
batch.jobs[0].results[0].metrics

{'test': {'roc_auc': 0.9933333333333333,
  'accuracy': 0.9666666666666667,
  'precision': 0.9696969696969696,
  'recall': 0.9666666666666667,
  'f1': 0.9665831244778613,
  'loss': 0.10969173908233643},
 'validation': {'roc_auc': 1.0,
  'accuracy': 0.9444444444444444,
  'precision': 0.9523809523809523,
  'recall': 0.9444444444444444,
  'f1': 0.9440559440559441,
  'loss': 0.09142466634511948},
 'train': {'roc_auc': 0.9988465974625144,
  'accuracy': 0.9705882352941176,
  'precision': 0.9708513708513707,
  'recall': 0.9705882352941176,
  'f1': 0.9705818732424834,
  'loss': 0.06465762108564377}}

## 11. Metrics & Visualization

For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.